# 画像認識練習

In [1]:
#basement module
import chainer
#import cupy
#インストールが　macではできない。慎重に。
import chainercv
import matplotlib
import numpy as np

import chainer.links as Link
import chainer.functions as Function


print(chainercv.__version__)
print(matplotlib.__version__)


0.11.0
3.0.2


必要となるデータセットは事前に用意してある。

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image

In [3]:
img = np.asarray(Image.open("train/image/000.png"))
label = np.asarray(Image.open("train/label/000.png"))

In [4]:
import glob
from chainer import datasets

def create_dataset(img_filenames,label_filenames):
    img = datasets.ImageDataset(img_filenames)
    img = datasets.TransformDataset(img,lambda x: x/255.)
    label = datasets.ImageDataset(label_filenames,dtype=np.int32)
    dataset = datasets.TupleDataset(img,label)
    return dataset


In [5]:
def create_datasets():
    train_img_filenames = sorted(glob.glob("train/image/*.png"))
    train_label_filenames = sorted(glob.glob("train/label/*.png"))
    train = create_dataset(train_img_filenames,train_label_filenames)
    
    val_img_filenames = sorted(glob.glob("val/image/*.png"))
    val_label_filenames = sorted(glob.glob("val/label/*.png"))
    val = create_dataset(val_img_filenames,val_label_filenames)
    return train,val

In [6]:
train,val = create_datasets()

In [7]:
print(len(train))
print(len(val))


234
26


In [8]:
class MultiLayerPerceptron(chainer.Chain):
    def __init__(self,out_h,out_w):
        super().__init__()
        with self.init_scope():
            self.l1 = Link.Linear(None,100)
            self.l2 = Link.Linear(100,100)
            self.l3 = Link.Linear(100,out_h*out_w)
        self.out_h = out_h
        self.out_w = out_w
        
    def forward(self,x):
        h = Function.relu(self.l1(x))
        h = Function.relu(self.l2(h))
        h = self.l3(h)
        n = x.shape[0]
        
        return h.reshape((n,1,self.out_h,self.out_w))

In [9]:
from chainer import iterators
from chainer import training
from chainer import optimizers
from chainer.training import extensions

def create_trainer(batchsize,train,val,stop,device=-1):
    model = MultiLayerPerceptron(out_h=256,out_w=256)
    train_model = Link.Classifier(model,lossfun=Function.sigmoid_cross_entropy,accfun=Function.binary_accuracy)
    optimizer = optimizers.Adam()
    optimizer.setup(train_model)
    
    train_iter = iterators.MultiprocessIterator(train,batchsize)
    val_iter = iterators.MultiprocessIterator(val,batchsize,repeat=False,shuffle=False)
    
    updater = training.StandardUpdater(train_iter,optimizer,device=device)
    trainer = training.trainer.Trainer(updater,stop)
    
    return trainer

In [10]:
%%time
trainer = create_trainer(64,train,val,(20,"epoch"),device=-1)
trainer.run()

CPU times: user 45.9 s, sys: 8.63 s, total: 54.6 s
Wall time: 33.3 s
